# Final Project GEOG5543 Part 1 Extract Interaction Data
The machine learning method k means clustering algorithm and density-based spatial clustering of applications with noise (DBSCAN) method will be used to group sets of latitude and longitude coordinate points from Travel Behavior Inventory (TBI) Household Survey Interview Data for 2010 in Minnesota to determine the clustering pattern of individual interaction (trip destination) location. The interaction between survey participants is defined by the time geography concept and space-time coupling constraints. The latitude and longitude coordinate points will be grouped into k groups using the unsupervised k means algorithm and it will be tested in DBSCAN to compare the results of these two different methods. This project will combine spatial analysis with the power of machine learning to determine and obtain the clustering pattern of TBI data to understand how the trip destination locations are distributed spatially and to test the performance of two different clustering methods.

## Define Potential Interaction
The potential interaction is defined and calculated between two persons using space-time coupling constraints, that is, if and only if two persons are present at the same location during the same time can they potentially interact. An individual is exposed to interaction with others in each location he or she visits. If person A and person B spend time at two different locations, there is no potential interaction between them. The interaction starts when they are presented at the same location at the same time. The interaction stops once either one of them exit the location. 


## Extract Interaction Data
In this notebook, Python is used to clean and process the merged table from TBI data. Python code is used to ignore the null value and missing data and then to identify participants that were presented at the same location at the same period of time and extract their activity information into new tables. 


The data is first sorted by ID, start time and end time of the trip so that they are listed as consecutive trips that are recorded from the same person. And the start time and end time of each trip are compared to identified two persons that are presented at the same location during the same time so if there is overlapping time period spend at the same location between two individuals, there is potential interaction between them based on the definition. The GPS locations, day of the week, person ID is extracted and further separated into five days of the week days, that is, Monday, Tuesday, Wednesday, Thursday, and Friday.

After the latitude and longitude data is ready in CSV format, the k means and DBSCAN will be applied to the data and generate a clustering point map in Folium map with dropdown widget in Part 2(second notebook)


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import csv
import datetime as dt

In [4]:
pd.options.mode.chained_assignment = None  # default='warn'

In [5]:
#Read in the merged data
df = pd.read_csv('Merge.csv')

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (20,26,28,51,52,57,127,128,190,193,199) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Read the trip record and insert activities between two consecutive trips; 
1. Ignore when trip origin and trip destination spatial unit is missing or null
2. Ignore when activity origin and destination spatial units is missing or null
3. Use the destination description of the previous trip to describe the following activity place
4. Identify all the person B that person A interact with because person A might interact with more than one person B at different times on different days 
        

In [24]:
# Extract selected column into new datafram df3
df3 = df[['HHPERSONID', 'TRIPID', 'HHID', 'HOMELAT','HOMELON','HTAZ',
                 'TRVDATE','DAY','OLAT','OLON','OTAZ', 'OLOCTYPE','OLOCACT1','LVLOCTIME','TRANSTYPE1',
                 'ARVTIME', 'DLAT','DLON','DTAZ','DLOCTYPE','DLOCACT1' , 
                 'origAct','destAct','mode1', 'arvtime_fin','lvloctime_fin',
                 'Gender','Student','Educ','Wrkr','Age3'
                 ]]

#convert start/end date and time into string with same format
df3['TRVDATE']= df3['TRVDATE'].astype('string')#Date
df3['lvloctime_fin'] = df3['lvloctime_fin'].astype('string')#start time
df3['lvloctime_fin'] = df3['lvloctime_fin'].str.zfill(4)#start time
df3['ARVTIME']=df3['ARVTIME'].astype('string')#end time
df3['ARVTIME'] = df3['ARVTIME'].str.zfill(4)#end time

#Create new variables to store Date + Time
df3['Start_DateTime'] = df3['TRVDATE']+ df3['lvloctime_fin']
df3['End_DateTime'] = df3['TRVDATE']+ df3['ARVTIME']
df3['Start_Time'] = pd.to_datetime(df3['Start_DateTime'],format ='%Y%m%d%H%M')#start time
df3['End_Time'] = pd.to_datetime(df3['End_DateTime'],format ='%Y%m%d%H%M')#end time

#Convert HHPERSONID into string
df3['HHPERSONID'] = df3['HHPERSONID'].astype('string')

In [25]:
df3.head()

,HHPERSONID,TRIPID,HHID,HOMELAT,HOMELON,HTAZ,TRVDATE,DAY,OLAT,OLON,...,lvloctime_fin,Gender,Student,Educ,Wrkr,Age3,Start_DateTime,End_DateTime,Start_Time,End_Time
0,10002101,1000210101,100021,44.954,-93.337,1374,20110610,6,44.954,-93.337,...,0830,Male,Not in school,Graduate/Post-graduate degree,A PAID part-time worker,7,201106100830,201106100945,2011-06-10 08:30:00,2011-06-10 09:45:00
1,10002101,1000210102,100021,44.954,-93.337,1374,20110610,6,44.954,-93.337,...,1030,Male,Not in school,Graduate/Post-graduate degree,A PAID part-time worker,7,201106101030,201106101035,2011-06-10 10:30:00,2011-06-10 10:35:00
2,10002101,1000210103,100021,44.954,-93.337,1374,20110610,6,44.954,-93.337,...,1040,Male,Not in school,Graduate/Post-graduate degree,A PAID part-time worker,7,201106101040,201106101045,2011-06-10 10:40:00,2011-06-10 10:45:00
3,10002101,1000210104,100021,44.954,-93.337,1374,20110610,6,44.954,-93.337,...,1145,Male,Not in school,Graduate/Post-graduate degree,A PAID part-time worker,7,201106101145,201106101155,2011-06-10 11:45:00,2011-06-10 11:55:00
4,10002101,1000210105,100021,44.954,-93.337,1374,20110610,6,44.968,-93.354,...,1320,Male,Not in school,Graduate/Post-graduate degree,A PAID part-time worker,7,201106101320,201106101330,2011-06-10 13:20:00,2011-06-10 13:30:00


In [26]:
# create a new column to store the activity sequences for each person

#Create an activity id 
df3['ACTIVITY_ID'] = np.nan

# sort the record by HHPERSONID and by Start Time and End Time
df3 = df3.sort_values(['HHPERSONID', 'Start_Time', 'End_Time'],ascending=[True, True, True])

# Group the data by 'HHPERSONID'
grouped = df3.groupby('HHPERSONID')

# create an empty dictionary for output
new_dict = {'HHPERSONID':[], 'TRIPID':[], 'ACTIVITY_ID':[], 
            'Start_Time':[], 'End_Time':[],
            'DAY':[], 'OLAT':[],
            'OLON':[], 'OTAZ':[],'DLAT':[],
            'DLON':[], 'DTAZ':[],
            'DLOCTYPE':[],'DLOCACT1':[],
           'Gender':[],'Student':[],'Educ':[],'Wrkr':[],'Age3':[]}

testCnt = 1
count =1
k = 0

#processing each person's record using for loop to loop through the grouped value
for name, group in grouped:
    k += 1

    totalCount = group.shape[0]# get the total number of records for that person
    activityCount = 0# get the total number of activity for that person to create TRIP_ID
    
    # insert the first activity episode; keep Person ID, update datetime and TAZ, keep others variables as nan
    act_row = group.iloc[0]
    new_dict['HHPERSONID'].append(act_row['HHPERSONID'])
    new_dict['TRIPID'].append(np.nan)
    new_dict['ACTIVITY_ID'].append(np.nan)
    tempDateTime = pd.to_datetime(act_row['Start_Time'].strftime('%Y-%m-%d') + ' 00:00:00')
    new_dict['Start_Time'].append(tempDateTime)
    new_dict['End_Time'].append(act_row['Start_Time'])
    new_dict['DLAT'].append(np.nan)
    new_dict['DLON'].append(np.nan)
    new_dict['OTAZ'].append(np.nan)
    new_dict['DTAZ'].append(act_row['OTAZ'])
    new_dict['Gender'].append(np.nan)
    new_dict['Student'].append(np.nan)
    new_dict['Educ'].append(np.nan)
    new_dict['Wrkr'].append(np.nan)
    new_dict['Age3'].append(np.nan)
    
    prev_row = group.iloc[0]
    for key in new_dict.keys():#add to dictionary
        new_dict[key].append(prev_row[key])
    next_row = None
    
    i = 1
    
    # if i is less than the total number of records, iterately update the dictionary
    while i < totalCount:
        next_row = group.iloc[i]
        prev_row = group.iloc[i-1]
        new_dict['HHPERSONID'].append(next_row['HHPERSONID'])
        new_dict['TRIPID'].append(np.nan)
        new_dict['Start_Time'].append(prev_row['End_Time'])
        new_dict['End_Time'].append(next_row['Start_Time'])
        new_dict['DLAT'].append(prev_row['DLAT'])
        new_dict['DLON'].append(prev_row['DLON'])
        new_dict['OTAZ'].append(prev_row['OTAZ'])
        new_dict['DTAZ'].append(next_row['DTAZ'])
        new_dict['Gender'].append(next_row['Gender'])
        new_dict['Student'].append(next_row['Student'])
        new_dict['Educ'].append(next_row['Educ'])
        new_dict['Wrkr'].append(next_row['Wrkr'])
        new_dict['Age3'].append(next_row['Age3'])
        # if the "to time(end time)" from previous row is less than the "from time(start time)" of the next row,(to keep it consecutive) 
        # update the activity ID by combining HHPERSONID with the number of activity of that person
        #(if not, keep it as null)
        if (prev_row['End_Time'] < next_row['Start_Time']):
            activityCount += 1
            cur_activityID = next_row['HHPERSONID'] + str(activityCount).zfill(3)
            new_dict['ACTIVITY_ID'].append(cur_activityID)
        else:
            new_dict['ACTIVITY_ID'].append(np.nan)
        
        for key in new_dict.keys():
            new_dict[key].append(next_row[key])
        #update i
        i += 1
        
    # update the rest of the record in the dictionary
    if next_row is None:
        # if next_row is none which means that there is only one record, set it as the previous
        # else set it as the next row
        act_row = prev_row
    else:
        act_row = next_row
    
    #append all the rows to the empty dictionary new_dict
    new_dict['HHPERSONID'].append(act_row['HHPERSONID'])
    new_dict['TRIPID'].append(np.nan)
    new_dict['ACTIVITY_ID'].append(np.nan)
    new_dict['Start_Time'].append(act_row['End_Time'])
    tempDateTime = pd.to_datetime(act_row['End_Time'].strftime('%Y-%m-%d') + ' 23:59:00')
    new_dict['End_Time'].append(tempDateTime)
    new_dict['DLAT'].append(act_row['DLAT'])
    new_dict['DLON'].append(act_row['DLAT'])
    new_dict['OTAZ'].append(act_row['OTAZ'])
    new_dict['DTAZ'].append(np.nan)
    new_dict['Gender'].append(act_row['Gender'])
    new_dict['Student'].append(act_row['Student'])
    new_dict['Educ'].append(act_row['Educ'])
    new_dict['Wrkr'].append(act_row['Wrkr'])
    new_dict['Age3'].append(act_row['Age3'])
print('>>> Finish processing')


>>> Finish processing


In [37]:
#convert dictionary to dataframe and export it to csv file
new_df=pd.DataFrame.from_dict(new_dict,orient='index').transpose()
new_df=pd.DataFrame(dict([(k,pd.Series(v))for k,v in new_dict.items()]))

In [39]:
#Export to CSV
new_df.to_csv('Activity.csv', index=False, 
            columns=['HHPERSONID','TRIPID','ACTIVITY_ID','Start_Time','End_Time','DAY','OLAT','OLON','OTAZ','DLAT','DLON', 'DTAZ','DLOCTYPE','DLOCACT1','Gender','Student','Educ','Wrkr','Age3'
                    ])

### Find potential interactiion and break down trip records into five days (Monday to Friday)
Monday 0, Tuesday 1, Wednesday 2, Thursday 3, Friday 4

And the start time and end time of each trip are compared to identified two persons that are presented at the same location during the same time so if there is overlapping time period spend at the same location between two individuals, there is potential interaction between them based on the definition

In [44]:

#Read in the csv file that is generated in above cell
df4 = pd.read_csv("Activity.csv")

# change the data type of the datetime
dt_format = '%Y-%m-%d %H:%M:%S'
df4['Start_Time'] = pd.to_datetime(df4['Start_Time'], format = dt_format)
df4['End_Time'] = pd.to_datetime(df4['End_Time'], format = dt_format)

# sort the data 
df4 = df4.sort_values(['HHPERSONID', 'Start_Time', 'End_Time'],ascending=[True, True, True])

# day of the week, with Monday=0, Tuesday 1, Wednesday 2, Thursday 3, Friday 4
df4['day_of_week'] = df4['Start_Time'].dt.dayofweek


In [45]:

# select only the activities that occured in the same TAZ and format columns
df4 = df4[~df4['ACTIVITY_ID'].isnull()]
df4 = df4.query('OTAZ == DTAZ')
df4['Start_Time'] = pd.to_datetime(df4['Start_Time'])
df4['End_Time'] = pd.to_datetime(df4['End_Time'])
df4['day_of_week'] =df4['day_of_week'].astype(str)


In [51]:
# group by day of the week
DOW_grouped = df4.groupby('day_of_week')
testCnt = 0
tempDate = dt.date.today()


#for all the day of the week (monday to friday in this case)
#processing each record using for loop to loop through the grouped value
for DOW, DOW_group in DOW_grouped:
    TAZ_grouped = DOW_group.groupby('OTAZ')
    print(">>> Now start to process weekday:", DOW) 
    
    newList = [['Day_of_Week','TAZ_ID','DLAT','DLON','StartTime','EndTime', 'Total_Sec', 'PersonID_1', 'ActivityID_1', 'PersonID_2', 'ActivityID_2','Gender','Student','Educ','Wrkr','Age']]
    for TAZ, TAZ_df in TAZ_grouped:
        total_cnt = len(TAZ_df)
        #if there is more than 2 records then start the comparision of start time and end time 
        if total_cnt >= 2:
            for i in range(0, total_cnt-1):
                line01 = TAZ_df.iloc[i]
                frTime01 = line01['Start_Time']
                toTime01 = line01['End_Time']
                for j in range(i+1, total_cnt):
                    line02 = TAZ_df.iloc[j]
                    frTime02 = line02['Start_Time']
                    toTime02 = line02['End_Time']
                    #find the max "from time(start time)" and min "to time(end time)" between two records
                    #and calculate their total interaction seconds
                    maxfrTime = max(frTime01, frTime02)
                    mintoTime = min(toTime01, toTime02)
                    durSeconds = int((mintoTime - maxfrTime).total_seconds())
                    # if total interaction seconds greater than 0 then extract the trip data into new row 
                    if durSeconds > 0:
                        newRow = [DOW,TAZ,line01['DLAT'],line01['DLON'],maxfrTime.time().strftime("%H:%M:%S"), mintoTime.time().strftime("%H:%M:%S"), durSeconds,
                                line01['HHPERSONID'], line01['ACTIVITY_ID'],line02['HHPERSONID'],line02['ACTIVITY_ID'],line01['Gender'],line01['Student'],line01['Educ'],line01['Wrkr'],line01['Age3']]
                        newList.append(newRow)
                      
    #append all the new row into new dataframe  
    new_df = pd.DataFrame(newList[1:],columns=newList[0])   
    newFileName = 'Interaction_ByDOW_'+str(int(DOW))+ '.csv'
   
    new_df.to_csv(newFileName, index=False)
    print(f">>> Finish writing the output data {newFileName}")             

print("Finished")
        

>>> Now start to process weekday: 0
>>> Finish writing the output data Interaction_ByDOW_0.csv
>>> Now start to process weekday: 1
>>> Finish writing the output data Interaction_ByDOW_1.csv
>>> Now start to process weekday: 2
>>> Finish writing the output data Interaction_ByDOW_2.csv
>>> Now start to process weekday: 3
>>> Finish writing the output data Interaction_ByDOW_3.csv
>>> Now start to process weekday: 4
>>> Finish writing the output data Interaction_ByDOW_4.csv
Finished


In [55]:
#test = pd.read_csv('Interaction_ByDOW_2.csv',encoding="ISO-8859-1")
#test#['Gender'].value_counts(normalize=True)*100